# TFRecords to Dataset
### This is work to convert .tfrecord files into Datasets, typically used in training models

In [ ]:
from Base_Deeplearning_Code.Data_Generators.TFRecord_to_Dataset_Generator import DataGeneratorClass as DataGenerator
from Base_Deeplearning_Code.Data_Generators.Plot_And_Scroll_Images.Plot_Scroll_Images import plot_Image_Scroll_Bar_Image

In [ ]:
record_path = r'K:\Morfeus\YHe\Brian\deep learning nifti debug\records'

In [ ]:
generator = DataGenerator(record_paths=[record_path])

### How many records do we have?

In [ ]:
len(generator) # The number of unique 2D slices

### Lets pull one of the examples

In [ ]:
data_set = generator.data_set

### To get a better understanding of datasets, please review
### https://blog.tensorflow.org/2019/02/introducing-tensorflow-datasets.html

In [ ]:
example = next(iter(data_set))

In [ ]:
example.keys()

In [ ]:
plot_Image_Scroll_Bar_Image(example['image_array'])

## Preparing data for a model
For many things, the benefit of datasets is the ability to make on the fly perturbations

### Dataset processors

In [ ]:
import Base_Deeplearning_Code.Image_Processors_Module.src.Processors.TFDataSetProcessors as DatasetProcessors

In [ ]:
processors = [
    DatasetProcessors.ExpandDimension(image_keys=('image_array', 'annotation_array'), axis=-1),
    DatasetProcessors.ToCategorical(annotation_keys=('annotation_array',), number_of_classes=(2,))
]

In [ ]:
for p in processors:
    example = p.parse(example)

In [ ]:
example['annotation_array'].shape

# Training a model

## Tracking hyperparameters

### Pause! Before we do anything, we need to make sure we will have an accurate, and reproducible record of what EXACTLY goes into each model. This could be the number of layers in the UNet, the loss function, the learning rate, or ANYTHING
### We want to create an excel sheet that will track each parameter to ensure that we can reproduce our model
### We will make the FIRST parameter be our 'Model_Index' this will iterate upwards by 1 and be our way of tracking each model made

In [ ]:
import pandas as pd
import os
model_excel_path = os.path.join('.', 'Model_Parameters.xlsx')

In [ ]:
hyper_parameters = ('Layers', 'Filters', 'Convs_Per_Block', 'Minimum_LR', 'Maximum_LR', 'Loss_Function')
metrics = ('Loss',)

In [ ]:
columns = ('Model_Index',) + hyper_parameters + metrics

In [ ]:
if not os.path.exists(model_excel_path):
    data_dict = {}
    for key in columns:
        data_dict[key] = []
    df = pd.DataFrame(data_dict)
    df.to_excel(model_excel_path, index=0, engine='openpyxl')

### Remember, you can always add more columns in later, just make sure that they are accounted for

## Lets make a model
### Don't forget to pass along the hyper parameters that you will need to fill in

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers

In [ ]:
def conv_block(x, Filters, blocks=2):
    Filters = int(Filters)
    for _ in range(blocks):
        x = layers.Conv2D(Filters, (3, 3), padding='Same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('selu')(x)
    return x
def return_model(Layers, Filters, Convs_Per_Block=2):
    img_input = x =layers.Input(shape=(512, 512, 1))
    concat_values = []
    for i in range(Layers - 1):
        x = conv_block(x, Filters, blocks=Convs_Per_Block)
        concat_values.append(x)
        x = layers.MaxPool2D()(x)
        Filters *= 2
    x = conv_block(x, Filters, blocks=Convs_Per_Block)
    for x_i in concat_values[::-1]:
        Filters /= 2
        x = layers.UpSampling2D()(x)
        x = conv_block(x, Filters, blocks=1)
        x = layers.Concatenate()([x, x_i])
        x = conv_block(x, Filters, blocks=Convs_Per_Block)
    x = conv_block(x, Filters, blocks=1)
    x = layers.Conv2D(2, (1, 1), padding='Same')(x)
    output = layers.Activation('softmax')(x)
    model = tf.keras.models.Model(inputs=(img_input,), outputs = (output,))
    return model

In [ ]:
model_dict = {'Layers': 2, 'Filters': 16, 'Convs_Per_Block': 2}
x = return_model(**model_dict)

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
help(plot_model)

In [ ]:
plot_model(model=x, to_file=os.path.join('.', 'model.png'), show_shapes=True, show_dtype=True,
           show_layer_names=False, rankdir='TB', dpi=96)